# 03 — Enhanced Two-Step Floating Catchment Area (E2SFCA) Analysis

Compute spatial accessibility scores for every Pennsylvania census tract
using the E2SFCA method with variable catchment sizes and distance decay.

**Outputs per tract:**
- Spatial accessibility score (PCPs per 1,000 pop, decay-weighted)
- Minimum drive-time to nearest facility
- Number of accessible facilities within catchment
- Provider-to-population ratio

In [ ]:
import sys
sys.path.insert(0, "..")

import geopandas as gpd
import pandas as pd

from src.config import DATA_RAW, DATA_PROCESSED, DATA_OUTPUTS
from src.spatial_analysis import (
    build_road_network,
    compute_drive_times,
    classify_urbanicity,
    e2sfca,
    sensitivity_analysis,
)
from src.visualization import choropleth_map

## 3.1 Load Processed Data

In [ ]:
tracts = gpd.read_file(DATA_PROCESSED / "pa_tracts_enriched.gpkg")
facilities = gpd.read_file(DATA_PROCESSED / "pa_facilities.gpkg")
roads = gpd.read_file(DATA_RAW / "tiger" / "tl_2024_42_roads.gpkg")

tracts.shape, facilities.shape, roads.shape

## 3.2 Build Road Network Graph

In [ ]:
road_graph = build_road_network(roads)
road_graph.number_of_nodes(), road_graph.number_of_edges()

## 3.3 Compute Pairwise Drive Times

In [ ]:
origins = tracts.copy()
origins["geometry"] = origins.geometry.centroid

drive_times = compute_drive_times(
    graph=road_graph,
    origins=origins,
    destinations=facilities,
    max_minutes=30.0,
)

drive_times.head(), drive_times.shape

## 3.4 Classify Tract Urbanicity

In [ ]:
tracts["urbanicity"] = classify_urbanicity(tracts).astype(str)
tracts[["geoid", "pop_density_sq_mi", "urbanicity"]].head()

## 3.5 Run E2SFCA (Baseline Parameters)

In [ ]:
e2sfca_out = e2sfca(
    tracts=tracts,
    facilities=facilities,
    drive_times=drive_times,
)

e2sfca_out[["geoid", "accessibility_score", "nearest_facility_min", "facilities_in_catchment", "provider_pop_ratio"]].head()

## 3.6 Sensitivity Analysis

In [ ]:
sensitivity_df = sensitivity_analysis(
    tracts=tracts,
    facilities=facilities,
    drive_times=drive_times,
)

sensitivity_df.head()

## 3.7 Initial Accessibility Maps

In [ ]:
access_map = choropleth_map(
    tracts=e2sfca_out,
    value_col="accessibility_score",
    title="Accessibility Score (E2SFCA)",
)
access_map

## 3.8 Save Accessibility Scores

In [ ]:
DATA_OUTPUTS.mkdir(parents=True, exist_ok=True)

e2sfca_path = DATA_OUTPUTS / "pa_accessibility_scores.gpkg"
drive_path = DATA_OUTPUTS / "pa_drive_times.csv"
sens_path = DATA_OUTPUTS / "pa_e2sfca_sensitivity.csv"
map_path = DATA_OUTPUTS / "pa_accessibility_map.html"

e2sfca_out.to_file(e2sfca_path, driver="GPKG")
drive_times.to_csv(drive_path, index=False)
sensitivity_df.to_csv(sens_path, index=False)
access_map.save(str(map_path))

e2sfca_path, drive_path, sens_path, map_path